In [ ]:
pip install pandas opencv-python matplotlib


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os

image_folder = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/test_results"
output_crops_folder = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_crop"
os.makedirs(output_crops_folder, exist_ok=True)

# Function to manually process detections (mock bounding box detection if not stored)
def detect_and_crop(image_path, output_folder, filename):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not load image: {filename}")
        return

    # Mock bounding box (example coordinates - replace with actual detection pipeline)
    # These should come from your model if not already stored.
    height, width, _ = image.shape
    mock_bboxes = [
        [int(0.1 * width), int(0.1 * height), int(0.3 * width), int(0.4 * height)],  # Example bounding box
        [int(0.5 * width), int(0.2 * height), int(0.8 * width), int(0.6 * height)],  # Example bounding box
    ]

    # Process and save each bounding box
    for i, bbox in enumerate(mock_bboxes):
        x1, y1, x2, y2 = bbox
        cropped = image[y1:y2, x1:x2]
        crop_filename = f"{os.path.splitext(filename)[0]}_crop_{i}.jpg"
        crop_path = os.path.join(output_folder, crop_filename)
        cv2.imwrite(crop_path, cropped)

for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Supported formats
        image_path = os.path.join(image_folder, filename)
        detect_and_crop(image_path, output_crops_folder, filename)

print(f"Cropping completed! Cropped images saved in {output_crops_folder}")


Cropping completed! Cropped images saved in C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_crop


In [ ]:
import os
import shutil

# Define paths
image_folder = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_crop'
new_folder = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_crop1'

# Create the new folder if it doesn't exist
os.makedirs(new_folder, exist_ok=True)

# Iterate over all files in the source folder
for image_name in os.listdir(image_folder):
    # Process only .jpg files
    if image_name.endswith('.jpg') and 'crop_1' in image_name:
        # Construct the new name by removing '_crop_1'
        new_name = image_name.replace('_crop_1', '')
        
        # Full paths
        old_path = os.path.join(image_folder, image_name)
        new_path = os.path.join(new_folder, new_name)
        
        # Copy the image to the new folder with the new name
        shutil.copy(old_path, new_path)
        print(f"Copied and renamed: {image_name} -> {new_name}")  # Log the copy (optional)

print("Crop_1 images copied and renamed successfully.")


Copied and renamed: 2023-06-28_17-31-45_crop_1.jpg -> 2023-06-28_17-31-45.jpg
Copied and renamed: 2023-06-28_17-51-48_crop_1.jpg -> 2023-06-28_17-51-48.jpg
Copied and renamed: 2023-06-28_18-11-48_crop_1.jpg -> 2023-06-28_18-11-48.jpg
Copied and renamed: 2023-06-28_18-31-48_crop_1.jpg -> 2023-06-28_18-31-48.jpg
Copied and renamed: 2023-06-28_18-51-49_crop_1.jpg -> 2023-06-28_18-51-49.jpg
Copied and renamed: 2023-06-29_06-00-16_crop_1.jpg -> 2023-06-29_06-00-16.jpg
Copied and renamed: 2023-06-29_06-27-31_crop_1.jpg -> 2023-06-29_06-27-31.jpg
Copied and renamed: 2023-06-29_06-47-33_crop_1.jpg -> 2023-06-29_06-47-33.jpg
Copied and renamed: 2023-06-29_07-07-33_crop_1.jpg -> 2023-06-29_07-07-33.jpg
Copied and renamed: 2023-06-29_07-27-33_crop_1.jpg -> 2023-06-29_07-27-33.jpg
Copied and renamed: 2023-06-29_07-47-33_crop_1.jpg -> 2023-06-29_07-47-33.jpg
Copied and renamed: 2023-06-29_08-07-34_crop_1.jpg -> 2023-06-29_08-07-34.jpg
Copied and renamed: 2023-06-29_08-37-33_crop_1.jpg -> 2023-06-29

In [1]:
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Directory containing images
image_dir = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_crop1"

# Function to preprocess an image
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")  # Open image as RGB
    img_tensor = F.to_tensor(img)  # Convert to PyTorch tensor
    return img, img_tensor


c:\Users\mjaga\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mjaga\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# Function to detect humans in an image
def detect_humans(model, image_tensor, confidence_threshold=0.5):
    with torch.no_grad():
        predictions = model([image_tensor])  # Model expects a list of tensors

    # Extract boxes, labels, and scores
    boxes = predictions[0]['boxes'].numpy()
    labels = predictions[0]['labels'].numpy()
    scores = predictions[0]['scores'].numpy()

    # Filter for "person" class (class ID 1 in COCO) and confidence threshold
    person_boxes = [boxes[i] for i in range(len(labels)) if labels[i] == 1 and scores[i] > confidence_threshold]
    return person_boxes

# Function to draw bounding boxes on an image
def draw_boxes(image, boxes):
    img_copy = np.array(image).copy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
    return img_copy


In [3]:
# Create output directory for results
output_dir = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_rcnn_crop1.1"
os.makedirs(output_dir, exist_ok=True)

# Process each image in the folder
results = []

for image_file in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_file)

    # Ensure the file is an image
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        print(f"Processing {image_file}...")
        
        # Preprocess and detect humans
        original_image, image_tensor = preprocess_image(image_path)
        person_boxes = detect_humans(model, image_tensor, confidence_threshold=0.5)
        
        # Draw boxes on the image
        output_image = draw_boxes(original_image, person_boxes)
        
        # Save the output image
        output_path = os.path.join(output_dir, image_file)
        Image.fromarray(output_image).save(output_path)
        
        # Save detection results
        results.append({"file": image_file, "human_count": len(person_boxes)})

# Print summary of results
for result in results:
    print(f"Image: {result['file']}, Humans Detected: {result['human_count']}")


Processing 2023-06-28_17-31-45.jpg...
Processing 2023-06-28_17-51-48.jpg...
Processing 2023-06-28_18-11-48.jpg...
Processing 2023-06-28_18-31-48.jpg...
Processing 2023-06-28_18-51-49.jpg...
Processing 2023-06-29_06-00-16.jpg...
Processing 2023-06-29_06-27-31.jpg...
Processing 2023-06-29_06-47-33.jpg...
Processing 2023-06-29_07-07-33.jpg...
Processing 2023-06-29_07-27-33.jpg...
Processing 2023-06-29_07-47-33.jpg...
Processing 2023-06-29_08-07-34.jpg...
Processing 2023-06-29_08-37-33.jpg...
Processing 2023-06-29_08-57-33.jpg...
Processing 2023-06-29_09-17-33.jpg...
Processing 2023-06-29_09-37-33.jpg...
Processing 2023-06-29_09-57-33.jpg...
Processing 2023-06-29_10-17-33.jpg...
Processing 2023-06-29_10-37-33.jpg...
Processing 2023-06-29_10-57-33.jpg...
Processing 2023-06-29_11-17-33.jpg...
Processing 2023-06-29_11-47-33.jpg...
Processing 2023-06-29_12-07-33.jpg...
Processing 2023-06-29_12-37-33.jpg...
Processing 2023-06-29_12-57-33.jpg...
Processing 2023-06-29_13-27-33.jpg...
Processing 2

In [4]:
import detectron2
print(detectron2.__version__)


0.6


In [5]:
import os
import cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Set up Detectron2 configuration for CPU
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold for detection
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
cfg.MODEL.DEVICE = "cpu"  # Use CPU

# Set up the predictor
predictor = DefaultPredictor(cfg)

# Define the input directory and output directory
input_dir = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_rcnn_crop1.1'  # Path to your 4000 images
output_dir = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron'  # Path where to save results

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all the images in the input directory
for image_name in os.listdir(input_dir):
    # Process only image files (you can add more checks if needed)
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_dir, image_name)
        
        # Read the image
        image = cv2.imread(image_path)
        
        # Perform inference on the image
        outputs = predictor(image)
        
        # Get the predicted instances
        instances = outputs["instances"]
        
        # Visualize the results (bounding boxes drawn)
        v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        output_image = v.draw_instance_predictions(instances.to("cpu")).get_image()[:, :, ::-1]
        
        # Save the output image with bounding boxes
        output_image_path = os.path.join(output_dir, f"detected_{image_name}")
        cv2.imwrite(output_image_path, output_image)
        print(f"Processed and saved: {output_image_path}")
        
# Done processing all images
print("Processing complete!")


c:\Users\mjaga\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-28_17-31-45.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-28_17-51-48.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-28_18-11-48.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-28_18-31-48.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-28_18-51-49.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-29_06-00-16.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron\detected_2023-06-29_06-27-31.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Ma

In [8]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime

# Define your HSV color ranges for RCNN and Detectron (you can adjust these values as needed)
rcnn_color_lower = np.array([0, 50, 50])  # Lower HSV for red
rcnn_color_upper = np.array([10, 255, 255])  # Upper HSV for red

detectron_color_lower = np.array([100, 50, 50])  # Lower HSV for blue
detectron_color_upper = np.array([130, 255, 255])  # Upper HSV for blue

# Parameters for bounding box filtering
min_area = 300  # Reduce minimum area to filter out very small boxes
max_area = 5000  # Limit maximum area to avoid large artifacts
aspect_ratio_range = (0.3, 3)  # Adjust aspect ratio filter to allow more flexibility
overlap_threshold = 0.4  # Lower NMS threshold to merge overlapping boxes more aggressively

# Path to your image directory
image_directory = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron"

# Path to the CSV output
csv_output_path = "output_bounding_boxes.csv"

# Function to apply morphological transformations to clean up the mask
def apply_morphology(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask

# Function for counting bounding boxes and drawing them on the image
def count_and_draw_bounding_boxes(image_path, lower_color, upper_color):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None, 0

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_image, lower_color, upper_color)
    mask = apply_morphology(mask)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        aspect_ratio = w / h if h != 0 else 0

        if min_area <= area <= max_area and aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]:
            bounding_boxes.append((x, y, w, h))

    bounding_boxes = non_max_suppression(np.array(bounding_boxes), overlap_threshold)
    
    return len(bounding_boxes)

# Non-Maximum Suppression to eliminate duplicate overlapping bounding boxes
def non_max_suppression(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:-1]])
        yy1 = np.maximum(y1[i], y1[idxs[:-1]])
        xx2 = np.minimum(x2[i], x2[idxs[:-1]])
        yy2 = np.minimum(y2[i], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))

    return boxes[pick].astype("int")

# Prepare CSV file
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "RCNN Bounding Boxes", "Detectron Bounding Boxes", "Timestamp"])

    # Process all images in the directory
    for image_name in os.listdir(image_directory):
        if image_name.endswith(".jpg"):
            image_path = os.path.join(image_directory, image_name)

            # Count RCNN and Detectron bounding boxes
            rcnn_count = count_and_draw_bounding_boxes(image_path, rcnn_color_lower, rcnn_color_upper)
            detectron_count = count_and_draw_bounding_boxes(image_path, detectron_color_lower, detectron_color_upper)

            # Extract the timestamp from the file name (assuming format "detected_YYYY-MM-DD_HH-MM-SS.jpg")
            timestamp = image_name.split('_')[1] + "_" + image_name.split('_')[2]

            # Write the data to the CSV file
            writer.writerow([image_name, rcnn_count, detectron_count, timestamp])

print("CSV file has been written successfully.")


CSV file has been written successfully.


In [9]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime

# Define your HSV color ranges for RCNN and Detectron (you can adjust these values as needed)
rcnn_color_lower = np.array([0, 50, 50])  # Lower HSV for red
rcnn_color_upper = np.array([10, 255, 255])  # Upper HSV for red

detectron_color_lower = np.array([100, 50, 50])  # Lower HSV for blue
detectron_color_upper = np.array([130, 255, 255])  # Upper HSV for blue

# Parameters for bounding box filtering
min_area = 300  # Reduce minimum area to filter out very small boxes
max_area = 5000  # Limit maximum area to avoid large artifacts
aspect_ratio_range = (0.3, 3)  # Adjust aspect ratio filter to allow more flexibility
overlap_threshold = 0.4  # Lower NMS threshold to merge overlapping boxes more aggressively

# Path to your image directory
image_directory = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/output_detectron"

# Path to the CSV output
csv_output_path = "output_bounding_boxes1.csv"

# Function to apply morphological transformations to clean up the mask
def apply_morphology(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask

# Function for counting bounding boxes and drawing them on the image
def count_and_draw_bounding_boxes(image_path, lower_color, upper_color):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None, 0

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_image, lower_color, upper_color)
    mask = apply_morphology(mask)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        aspect_ratio = w / h if h != 0 else 0

        if min_area <= area <= max_area and aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]:
            bounding_boxes.append((x, y, w, h))

    bounding_boxes = non_max_suppression(np.array(bounding_boxes), overlap_threshold)
    
    return len(bounding_boxes)

# Non-Maximum Suppression to eliminate duplicate overlapping bounding boxes
def non_max_suppression(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:-1]])
        yy1 = np.maximum(y1[i], y1[idxs[:-1]])
        xx2 = np.minimum(x2[i], x2[idxs[:-1]])
        yy2 = np.minimum(y2[i], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))

    return boxes[pick].astype("int")

# Prepare CSV file
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "Total Bounding Boxes", "Timestamp"])

    # Process all images in the directory
    for image_name in os.listdir(image_directory):
        if image_name.endswith(".jpg"):
            image_path = os.path.join(image_directory, image_name)

            # Count RCNN and Detectron bounding boxes
            rcnn_count = count_and_draw_bounding_boxes(image_path, rcnn_color_lower, rcnn_color_upper)
            detectron_count = count_and_draw_bounding_boxes(image_path, detectron_color_lower, detectron_color_upper)

            # Sum the counts to get the total bounding boxes
            total_bounding_boxes = rcnn_count + detectron_count

            # Extract the timestamp from the file name (assuming format "detected_YYYY-MM-DD_HH-MM-SS.jpg")
            timestamp = image_name.split('_')[1] + "_" + image_name.split('_')[2]

            # Write the data to the CSV file
            writer.writerow([image_name, total_bounding_boxes, timestamp])

print("CSV file has been written successfully.")


CSV file has been written successfully.


In [2]:
import cv2
import os

# Function to crop the image into 4 parts and save them
def crop_image(image_path, output_folder):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Image {image_path} not found.")
        return
    
    # Get image dimensions (height and width)
    h, w = image.shape[:2]
    
    # Crop into 4 parts: top-left, top-right, bottom-left, bottom-right
    crops = {
        'top_left': image[0:h//2, 0:w//2],
        'top_right': image[0:h//2, w//2:w],
        'bottom_left': image[h//2:h, 0:w//2],
        'bottom_right': image[h//2:h, w//2:w]
    }

    # Process and save each cropped part
    for region, cropped_image in crops.items():
        # Create the filename for the cropped image
        filename = f"{os.path.basename(image_path).split('.')[0]}_{region}.jpg"
        output_path = os.path.join(output_folder, filename)

        # Save the cropped image
        cv2.imwrite(output_path, cropped_image)
        print(f"Saved cropped image: {output_path}")

# Function to process a folder of images and crop them
def process_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Process all images in the input folder
    for image_name in os.listdir(input_folder):
        if image_name.endswith(".jpg"):  # Adjust if you have other formats
            image_path = os.path.join(input_folder, image_name)
            
            # Crop the image into 4 parts and save them
            crop_image(image_path, output_folder)

# Example usage
input_folder = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/seletected _output"  # Replace with your input folder path
output_folder = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop"  # Replace with your output folder path

# Process the images in the input folder and save the cropped parts
process_images(input_folder, output_folder)


Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-37-33_top_left.jpg
Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-37-33_top_right.jpg
Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-37-33_bottom_left.jpg
Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-37-33_bottom_right.jpg
Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-57-33_top_left.jpg
Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-57-33_top_right.jpg
Saved cropped image: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop\2023-06-29_10-57-33_bottom_left.jpg
Saved cropped i

In [3]:
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Directory containing images
image_dir = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_crop"

# Function to preprocess an image
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")  # Open image as RGB
    img_tensor = F.to_tensor(img)  # Convert to PyTorch tensor
    return img, img_tensor


c:\Users\mjaga\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mjaga\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Function to detect humans in an image
def detect_humans(model, image_tensor, confidence_threshold=0.5):
    with torch.no_grad():
        predictions = model([image_tensor])  # Model expects a list of tensors

    # Extract boxes, labels, and scores
    boxes = predictions[0]['boxes'].numpy()
    labels = predictions[0]['labels'].numpy()
    scores = predictions[0]['scores'].numpy()

    # Filter for "person" class (class ID 1 in COCO) and confidence threshold
    person_boxes = [boxes[i] for i in range(len(labels)) if labels[i] == 1 and scores[i] > confidence_threshold]
    return person_boxes

# Function to draw bounding boxes on an image
def draw_boxes(image, boxes):
    img_copy = np.array(image).copy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
    return img_copy


In [5]:
# Create output directory for results
output_dir = "C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_rcnn"
os.makedirs(output_dir, exist_ok=True)

# Process each image in the folder
results = []

for image_file in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_file)

    # Ensure the file is an image
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        print(f"Processing {image_file}...")
        
        # Preprocess and detect humans
        original_image, image_tensor = preprocess_image(image_path)
        person_boxes = detect_humans(model, image_tensor, confidence_threshold=0.5)
        
        # Draw boxes on the image
        output_image = draw_boxes(original_image, person_boxes)
        
        # Save the output image
        output_path = os.path.join(output_dir, image_file)
        Image.fromarray(output_image).save(output_path)
        
        # Save detection results
        results.append({"file": image_file, "human_count": len(person_boxes)})

# Print summary of results
for result in results:
    print(f"Image: {result['file']}, Humans Detected: {result['human_count']}")


Processing 2023-06-29_10-37-33_bottom_left.jpg...
Processing 2023-06-29_10-37-33_bottom_right.jpg...
Processing 2023-06-29_10-37-33_top_left.jpg...
Processing 2023-06-29_10-37-33_top_right.jpg...
Processing 2023-06-29_10-57-33_bottom_left.jpg...
Processing 2023-06-29_10-57-33_bottom_right.jpg...
Processing 2023-06-29_10-57-33_top_left.jpg...
Processing 2023-06-29_10-57-33_top_right.jpg...
Processing 2023-06-29_11-17-33_bottom_left.jpg...
Processing 2023-06-29_11-17-33_bottom_right.jpg...
Processing 2023-06-29_11-17-33_top_left.jpg...
Processing 2023-06-29_11-17-33_top_right.jpg...
Processing 2023-06-29_11-47-33_bottom_left.jpg...
Processing 2023-06-29_11-47-33_bottom_right.jpg...
Processing 2023-06-29_11-47-33_top_left.jpg...
Processing 2023-06-29_11-47-33_top_right.jpg...
Processing 2023-06-29_12-07-33_bottom_left.jpg...
Processing 2023-06-29_12-07-33_bottom_right.jpg...
Processing 2023-06-29_12-07-33_top_left.jpg...
Processing 2023-06-29_12-07-33_top_right.jpg...
Processing 2023-06-2

In [6]:
import os
import cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Set up Detectron2 configuration for CPU
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold for detection
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
cfg.MODEL.DEVICE = "cpu"  # Use CPU

# Set up the predictor
predictor = DefaultPredictor(cfg)

# Define the input directory and output directory
input_dir = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_rcnn'  # Path to your 4000 images
output_dir = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron'  # Path where to save results

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all the images in the input directory
for image_name in os.listdir(input_dir):
    # Process only image files (you can add more checks if needed)
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_dir, image_name)
        
        # Read the image
        image = cv2.imread(image_path)
        
        # Perform inference on the image
        outputs = predictor(image)
        
        # Get the predicted instances
        instances = outputs["instances"]
        
        # Visualize the results (bounding boxes drawn)
        v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        output_image = v.draw_instance_predictions(instances.to("cpu")).get_image()[:, :, ::-1]
        
        # Save the output image with bounding boxes
        output_image_path = os.path.join(output_dir, f"detected_{image_name}")
        cv2.imwrite(output_image_path, output_image)
        print(f"Processed and saved: {output_image_path}")
        
# Done processing all images
print("Processing complete!")


c:\Users\mjaga\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron\detected_2023-06-29_10-37-33_bottom_left.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron\detected_2023-06-29_10-37-33_bottom_right.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron\detected_2023-06-29_10-37-33_top_left.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron\detected_2023-06-29_10-37-33_top_right.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron\detected_2023-06-29_10-57-33_bottom_left.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/beach test results/selected_output_detectron\detected_2023-06-29_10-57-33_bottom_right.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/b

In [1]:
import cv2
import os

# Folder containing the Faster R-CNN output images
image_folder = "jetty test results/test_results"
output_crops_folder = "C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_crop"
os.makedirs(output_crops_folder, exist_ok=True)

# Function to manually process detections (mock bounding box detection if not stored)
def detect_and_crop(image_path, output_folder, filename):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not load image: {filename}")
        return

    # Mock bounding box (example coordinates - replace with actual detection pipeline)
    # These should come from your model if not already stored.
    height, width, _ = image.shape
    mock_bboxes = [
        [int(0.1 * width), int(0.1 * height), int(0.3 * width), int(0.4 * height)],  # Example bounding box
        [int(0.5 * width), int(0.2 * height), int(0.8 * width), int(0.6 * height)],  # Example bounding box
    ]

    # Process and save each bounding box
    for i, bbox in enumerate(mock_bboxes):
        x1, y1, x2, y2 = bbox
        cropped = image[y1:y2, x1:x2]
        crop_filename = f"{os.path.splitext(filename)[0]}_crop_{i}.jpg"
        crop_path = os.path.join(output_folder, crop_filename)
        cv2.imwrite(crop_path, cropped)

# Loop through the Faster R-CNN output folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Supported formats
        image_path = os.path.join(image_folder, filename)
        detect_and_crop(image_path, output_crops_folder, filename)

print(f"Cropping completed! Cropped images saved in {output_crops_folder}")


Cropping completed! Cropped images saved in C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_crop


In [2]:
import os
import shutil

# Define paths
image_folder = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_crop'
new_folder = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_crop1'

# Create the new folder if it doesn't exist
os.makedirs(new_folder, exist_ok=True)

# Iterate over all files in the source folder
for image_name in os.listdir(image_folder):
    # Process only .jpg files
    if image_name.endswith('.jpg') and 'crop_1' in image_name:
        # Construct the new name by removing '_crop_1'
        new_name = image_name.replace('_crop_1', '')
        
        # Full paths
        old_path = os.path.join(image_folder, image_name)
        new_path = os.path.join(new_folder, new_name)
        
        # Copy the image to the new folder with the new name
        shutil.copy(old_path, new_path)
        print(f"Copied and renamed: {image_name} -> {new_name}")  # Log the copy (optional)

print("Crop_1 images copied and renamed successfully.")


Copied and renamed: 2023-06-28_17-31-42_crop_1.jpg -> 2023-06-28_17-31-42.jpg
Copied and renamed: 2023-06-28_17-51-45_crop_1.jpg -> 2023-06-28_17-51-45.jpg
Copied and renamed: 2023-06-28_18-11-46_crop_1.jpg -> 2023-06-28_18-11-46.jpg
Copied and renamed: 2023-06-28_18-41-45_crop_1.jpg -> 2023-06-28_18-41-45.jpg
Copied and renamed: 2023-06-29_06-00-16_crop_1.jpg -> 2023-06-29_06-00-16.jpg
Copied and renamed: 2023-06-29_06-28-02_crop_1.jpg -> 2023-06-29_06-28-02.jpg
Copied and renamed: 2023-06-29_06-48-03_crop_1.jpg -> 2023-06-29_06-48-03.jpg
Copied and renamed: 2023-06-29_07-08-03_crop_1.jpg -> 2023-06-29_07-08-03.jpg
Copied and renamed: 2023-06-29_07-28-04_crop_1.jpg -> 2023-06-29_07-28-04.jpg
Copied and renamed: 2023-06-29_07-48-08_crop_1.jpg -> 2023-06-29_07-48-08.jpg
Copied and renamed: 2023-06-29_08-18-03_crop_1.jpg -> 2023-06-29_08-18-03.jpg
Copied and renamed: 2023-06-29_08-38-03_crop_1.jpg -> 2023-06-29_08-38-03.jpg
Copied and renamed: 2023-06-29_08-58-03_crop_1.jpg -> 2023-06-29

In [6]:
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Directory containing images
image_dir = "C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_crop1"

# Function to preprocess an image
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")  # Open image as RGB
    img_tensor = F.to_tensor(img)  # Convert to PyTorch tensor
    return img, img_tensor


In [7]:
# Function to detect humans in an image
def detect_humans(model, image_tensor, confidence_threshold=0.5):
    with torch.no_grad():
        predictions = model([image_tensor])  # Model expects a list of tensors

    # Extract boxes, labels, and scores
    boxes = predictions[0]['boxes'].numpy()
    labels = predictions[0]['labels'].numpy()
    scores = predictions[0]['scores'].numpy()

    # Filter for "person" class (class ID 1 in COCO) and confidence threshold
    person_boxes = [boxes[i] for i in range(len(labels)) if labels[i] == 1 and scores[i] > confidence_threshold]
    return person_boxes

# Function to draw bounding boxes on an image
def draw_boxes(image, boxes):
    img_copy = np.array(image).copy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
    return img_copy


In [8]:
# Create output directory for results
output_dir = "C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_rcnn_crop1.1.1"
os.makedirs(output_dir, exist_ok=True)

# Process each image in the folder
results = []

for image_file in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_file)

    # Ensure the file is an image
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        print(f"Processing {image_file}...")
        
        # Preprocess and detect humans
        original_image, image_tensor = preprocess_image(image_path)
        person_boxes = detect_humans(model, image_tensor, confidence_threshold=0.5)
        
        # Draw boxes on the image
        output_image = draw_boxes(original_image, person_boxes)
        
        # Save the output image
        output_path = os.path.join(output_dir, image_file)
        Image.fromarray(output_image).save(output_path)
        
        # Save detection results
        results.append({"file": image_file, "human_count": len(person_boxes)})

# Print summary of results
for result in results:
    print(f"Image: {result['file']}, Humans Detected: {result['human_count']}")


Processing 2023-06-28_17-31-42.jpg...
Processing 2023-06-28_17-51-45.jpg...
Processing 2023-06-28_18-11-46.jpg...
Processing 2023-06-28_18-41-45.jpg...
Processing 2023-06-29_06-00-16.jpg...
Processing 2023-06-29_06-28-02.jpg...
Processing 2023-06-29_06-48-03.jpg...
Processing 2023-06-29_07-08-03.jpg...
Processing 2023-06-29_07-28-04.jpg...
Processing 2023-06-29_07-48-08.jpg...
Processing 2023-06-29_08-18-03.jpg...
Processing 2023-06-29_08-38-03.jpg...
Processing 2023-06-29_08-58-03.jpg...
Processing 2023-06-29_09-18-04.jpg...
Processing 2023-06-29_09-38-04.jpg...
Processing 2023-06-29_09-58-04.jpg...
Processing 2023-06-29_10-28-03.jpg...
Processing 2023-06-29_10-48-04.jpg...
Processing 2023-06-29_11-08-04.jpg...
Processing 2023-06-29_11-38-03.jpg...
Processing 2023-06-29_11-58-04.jpg...
Processing 2023-06-29_12-18-04.jpg...
Processing 2023-06-29_12-38-04.jpg...
Processing 2023-06-29_13-08-04.jpg...
Processing 2023-06-29_13-28-04.jpg...
Processing 2023-06-29_13-48-04.jpg...
Processing 2

In [9]:
import detectron2
print(detectron2.__version__)


0.6


In [11]:
import os
import cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Set up Detectron2 configuration for CPU
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold for detection
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
cfg.MODEL.DEVICE = "cpu"  # Use CPU

# Set up the predictor
predictor = DefaultPredictor(cfg)

# Define the input directory and output directory
input_dir = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_rcnn_crop1.1.1'  # Path to your 4000 images
output_dir = 'C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron'  # Path where to save results

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all the images in the input directory
for image_name in os.listdir(input_dir):
    # Process only image files (you can add more checks if needed)
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(input_dir, image_name)
        
        # Read the image
        image = cv2.imread(image_path)
        
        # Perform inference on the image
        outputs = predictor(image)
        
        # Get the predicted instances
        instances = outputs["instances"]
        
        # Visualize the results (bounding boxes drawn)
        v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        output_image = v.draw_instance_predictions(instances.to("cpu")).get_image()[:, :, ::-1]
        
        # Save the output image with bounding boxes
        output_image_path = os.path.join(output_dir, f"detected_{image_name}")
        cv2.imwrite(output_image_path, output_image)
        print(f"Processed and saved: {output_image_path}")
        
# Done processing all images
print("Processing complete!")


Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-28_17-31-42.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-28_17-51-45.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-28_18-11-46.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-28_18-41-45.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-29_06-00-16.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-29_06-28-02.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron\detected_2023-06-29_06-48-03.jpg
Processed and saved: C:/Users/mjaga/OneDrive/Desktop/Ma

In [13]:
import cv2
import numpy as np
import os
import csv
from datetime import datetime

# Define your HSV color ranges for RCNN and Detectron (you can adjust these values as needed)
rcnn_color_lower = np.array([0, 50, 50])  # Lower HSV for red
rcnn_color_upper = np.array([10, 255, 255])  # Upper HSV for red

detectron_color_lower = np.array([100, 50, 50])  # Lower HSV for blue
detectron_color_upper = np.array([130, 255, 255])  # Upper HSV for blue

# Parameters for bounding box filtering
min_area = 300  # Reduce minimum area to filter out very small boxes
max_area = 5000  # Limit maximum area to avoid large artifacts
aspect_ratio_range = (0.3, 3)  # Adjust aspect ratio filter to allow more flexibility
overlap_threshold = 0.4  # Lower NMS threshold to merge overlapping boxes more aggressively

# Path to your image directory
image_directory = "C:/Users/mjaga/OneDrive/Desktop/Main Project/jetty test results/output_detectron"

# Path to the CSV output
csv_output_path = "output_bounding_boxes1.1.1.csv"

# Function to apply morphological transformations to clean up the mask
def apply_morphology(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask

# Function for counting bounding boxes and drawing them on the image
def count_and_draw_bounding_boxes(image_path, lower_color, upper_color):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None, 0

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_image, lower_color, upper_color)
    mask = apply_morphology(mask)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        aspect_ratio = w / h if h != 0 else 0

        if min_area <= area <= max_area and aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]:
            bounding_boxes.append((x, y, w, h))

    bounding_boxes = non_max_suppression(np.array(bounding_boxes), overlap_threshold)
    
    return len(bounding_boxes)

# Non-Maximum Suppression to eliminate duplicate overlapping bounding boxes
def non_max_suppression(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:-1]])
        yy1 = np.maximum(y1[i], y1[idxs[:-1]])
        xx2 = np.minimum(x2[i], x2[idxs[:-1]])
        yy2 = np.minimum(y2[i], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))

    return boxes[pick].astype("int")

# Prepare CSV file
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "RCNN Bounding Boxes", "Detectron Bounding Boxes", "Timestamp"])

    # Process all images in the directory
    for image_name in os.listdir(image_directory):
        if image_name.endswith(".jpg"):
            image_path = os.path.join(image_directory, image_name)

            # Count RCNN and Detectron bounding boxes
            rcnn_count = count_and_draw_bounding_boxes(image_path, rcnn_color_lower, rcnn_color_upper)
            detectron_count = count_and_draw_bounding_boxes(image_path, detectron_color_lower, detectron_color_upper)

            # Extract the timestamp from the file name (assuming format "detected_YYYY-MM-DD_HH-MM-SS.jpg")
            timestamp = image_name.split('_')[1] + "_" + image_name.split('_')[2]

            # Write the data to the CSV file
            writer.writerow([image_name, rcnn_count, detectron_count, timestamp])

print("CSV file has been written successfully.")


CSV file has been written successfully.
